## Imports

In [6]:
import logging
import os
import shutil
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from datetime import datetime

from flask import Flask, request
app = Flask(__name__)

In [7]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


## Variables

In [8]:
file = "data.csv"
latest_pkl = "latest.pkl"

## Data Load/Transform

In [9]:
def etl(file):
    df01 = pd.read_csv(file)
    df01['Transaction Date'] = pd.to_datetime(df01['Transaction Date'], infer_datetime_format=True)
    df01.rename(columns={'Transaction Date':'Transaction_Date'}, inplace=True)

    data = df01.drop(['Transaction_Date'], axis=1)
    data.index = df01.Transaction_Date

    data01 = data[(data["ATM Name"]==1)]
    data01 = data01.sort_index()

    train = data01.astype(np.float64)
    logging.info("ETL completed.")
    return train

## Train Model

In [10]:
def VARMAX_Train_Model(train):
    exog = train[['Weekday','Festival Religion','Working Day','Holiday Sequence']]
    model= sm.tsa.VARMAX(train[['Total amount Withdrawn','Amount withdrawn XYZ Card']], order=(5,0), trend='c', exog=exog)
    model_result = model.fit(maxiter=1000, disp=False)
    logging.info("Model training completed.")
    return model_result

## Export Model

In [11]:
def exportModel(model_result):
    import pickle 
    from datetime import datetime
    
    #latest time now
    now = datetime.now()
    timestamp = now.strftime("%m-%d-%Y-%H-%M-%S")
    
    filename = 'ATM_VARMAX01_' + timestamp + '.pkl'
    pickle.dump(model_result, open(filename, 'wb'))
    logging.info("Model exported.")
    
    return filename;

def handleNewVersion(latestFileName):
    try:
        os.remove('latest.pkl')
    except:
        logging.info("Existing latest.pkl not found. Proceeding...")
    
    newPath = shutil.copy(latestFileName, 'latest.pkl')
    logging.info("Latest pkl file updated.")
    return 0;

## Forecasting Function

In [20]:
def forecast(Weekday,FestivalReligion,WorkingDay,HolidaySequence):
    # Use the loaded pickled model to make predictions 
    pkl_file = open(latest_pkl, 'rb')
    model_result = pickle.load(pkl_file) 
    params = []
#     test01 = test[['Weekday','Festival Religion','Working Day','Holiday Sequence']]
#     test01 = [[5.0,4.0,0.0,1.0]]
    test01 = [[Weekday,FestivalReligion,WorkingDay,HolidaySequence]]
    
    exog = test01
    pred = model_result.get_forecast(steps=1,exog=exog)
    pred_m = pred.predicted_mean
    logging.info("Prediction task completed.")
    
    totalAmount = pred_m["Total amount Withdrawn"] 
    XYZAmount = pred_m["Amount withdrawn XYZ Card"] 
    return [totalAmount,XYZAmount]

## Append New Data

In [13]:
def appendNewData(newData):
    f= open(file,"a+")
    if(checkNewLine()):
        f.write(newData + '\n')
    else:
        f.write('\n' + newData + '\n')
    f.close()
    logging.info("Appended new row -> \n" + newData)
    return 0;

def checkNewLine():
    outcome = False
    with open(file, 'rb+') as f:
        f.seek(-1,2)
        lastByte = f.read().decode("utf-8")
    if(lastByte == '\n'):
        outcome = True
    return outcome

## Workflow

In [14]:
processed_data = etl(file)
model_output = VARMAX_Train_Model(processed_data)
latestFileName = exportModel(model_output)
handleNewVersion(latestFileName)

INFO:root:ETL completed.
C:\Users\Kavish\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\Users\Kavish\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\representation.py:375: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return matrix[[slice(None)]*(matrix.ndim-1) + [0]]
INFO:root:Model training completed.
INFO:root:Model exported.
INFO:root:Existing latest.pkl not found. Proceeding...
INFO:root:Latest pkl file updated.


0

In [26]:
# output = forecast(1.0,1.0,1.0,1.0)

In [50]:
float(output[0].values)

420290.06333141215

## Helper Functions for Flask

In [10]:
required_length = 4

def validateNewData(data):
    valid = False
    args = data.split(',')
    if(len(args)) == required_length:
        valid = True
    return valid

## Flask Routes and Operations

In [12]:
# configure Flask routes

@app.route('/')
def index():
  return 'Flask service is up and running!'

@app.route('/addrow')
def addRow():
    data = request.args['data']
    appendNewData(data)
    return jsonify({"operation": "addrow"},{"status": "success"},{"data":data}), 200

@app.route('/updatemodel')
def updateModel():
    appendNewData(data)
    return jsonify({"operation": "updatemodel"},{"status": "success"}), 200